In [1]:
import pandas as pd # On importe la librairie Pandas, que l'on surnomme 'pd'
import numpy as np
import re

In [2]:
def lower_case(value): 
    print('Voici la valeur que je traite:', value)
    return value.lower()

In [3]:
data = pd.read_csv('personnes.csv', sep=',')
data.head()

,prenom,email,date_naissance,pays,taille
0,Leila,leila@example.com,23/01/1990,France,1.49m
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m


Il reste encore à savoir comment remplir  nouvelle_colonne. En fait, elle sera calculée à partir de  nom_colonne. Ce qu'il nous faut, c'est parcourir chaque valeur de  nom_colonne, vérifier si elle est correcte ou pas, et la corriger si besoin.

Pour cela, nous utilisons la méthode  apply. C'est une méthode qui est appelée sur une colonne du dataframe, et qui permet de parcourir toutes ses valeurs. Alternativement, on peut utiliser la méthode  map, qui lui est (à peu près) équivalente. Pour chaque valeur, elle applique une fonction destinée à la vérification/correction

In [4]:
prenom_min = data['prenom'].apply(lower_case)
prenom_min = prenom_min.values
print(prenom_min)
data['prenom_min'] = prenom_min

Voici la valeur que je traite: Leila
Voici la valeur que je traite: Samuel
Voici la valeur que je traite: Radia
Voici la valeur que je traite: Marc
Voici la valeur que je traite: Heri
Voici la valeur que je traite: Hanna
Voici la valeur que je traite: samuël
['leila' 'samuel' 'radia' 'marc' 'heri' 'hanna' 'samuël']


In [5]:
data

,prenom,email,date_naissance,pays,taille,prenom_min
0,Leila,leila@example.com,23/01/1990,France,1.49m,leila
1,Samuel,samuel_329@example.com,20/09/2001,NaN,1.67m,samuel
2,Radia,choupipoune@supermail.eu,12 sept. 1984,Côte d'ivoire,153cm,radia
3,Marc,"marco23@example.com, mc23@supermail.eu",10/02/1978,France,1.65m,marc
4,Heri,helloworld@supermail.eu,05/03/2008,Madagascar,1.34m,heri
5,Hanna,hanna2019@supermail.eu,01/01/1970,24,3.45m,hanna
6,samuël,samuel_329@example.com,NaN,Bénin,1.45m,samuël


### Traiter les pays

Ici, si le pays présent dans la variable  country  n'est pas présent dans la liste  VALID_COUNTRIES,  on affiche le message des lignes 6 et 7. Ensuite, on retourne  np.NaN, qui est la valeur utilisée par les librairies  Numpy  et  Pandas  pour spécifier qu'une valeur est inconnue. C'est en quelque sorte un équivalent de  None.

In [6]:
VALID_COUNTRIES = ['France', 'Côte d\'ivoire', 'Madagascar', 'Bénin', 'Allemagne'
                  , 'USA']
                  
def check_country(country):
    if country not in VALID_COUNTRIES:
        print(' - "{}" n\'est pas un pays valide, nous le supprimons.' \
            .format(country))
        return np.NaN
    return country

### Traiter les emails

Lorsqu'il y a plusieurs emails par ligne, ceux-ci sont séparés par des virgules. Nous séparons donc la chaîne de caractère de la variable  string  selon les virgules grâce à la méthode split (ligne 2). Le résultat est donc une liste avec autant d'éléments que d'adresses email: cette liste est placée dans la variable  parts.

Comme  parts  contient au moins 1 élément, on place celui-ci dans la variable  first_part. Ensuite, on compte le nombre d'éléments que contient  parts  grâce à la fonction  len. S'il y en a au moins 2, alors on affiche le message des lignes 5 et 6. Finalement, on retourne  first_part, qui contient le premier email !

In [8]:
def first(string):
    parts = string.split(',')
    first_part = parts[0]
    if len(parts) >= 2:
        print(' - Il y a plusieurs parties dans "{}", ne gardons que {}.'\
            .format(parts,first_part))  
    return first_part

### Traiter les tailles
Nous aurons ici 2 fonctions :  convert_height, qui convertira les chaînes de caractères de type  "1,34 m"  en nombre décimal, ainsi que  fill_height, qui remplacera les valeurs manquantes par la moyenne des tailles de l'échantillon.
La première fonction est un peu plus élaborée. Vous pouvez soit lui accorder une confiance aveugle, soit tenter de percer son mystère dans la section Aller plus loin en bas de ce chapitre ;).

Passons à la seconde fonction. Ah ! elle prend 2 paramètres :  height  et  replacement. Le premier est la hauteur, comme d'habitude. Le second est la valeur que nous devrons renvoyer en cas de valeur manquante. La ligne 11 vérifie si la valeur de  height  est manquante (None, NaN ou Nat). Si c'est le cas, on retourne la valeur de remplacement (ligne 13), sinon, on retourne  height.

In [9]:
def convert_height(height):
    found = re.search('\d\.\d{2}m', height)
    if found is None:
        print('{} n\'est pas au bon format. Il sera ignoré.'.format(height))
        return np.NaN
    else:
        value = height[:-1] # on enlève le dernier caractère, qui est 'm'
        return float(value)

def fill_height(height, replacement):
    if pd.isnull(height):
        print('Imputation par la moyenne : {}'.format(replacement))
        return replacement
    return height

### Appliquer les fonctions

In [10]:
data['email'] = data['email'].apply(first)
data['pays'] = data['pays'].apply(check_country)
data['taille'] = [convert_height(t) for t in data['taille']]
data['taille'] = [t if t<3 else np.NaN for t in data['taille']]
mean_height = data['taille'].mean()
data['taille'] = [fill_height(t, mean_height) for t in data['taille']]
data['date_naissance'] = pd.to_datetime(data['date_naissance'], 
                                           format='%d/%m/%Y', errors='coerce')


 - Il y a plusieurs parties dans "['marco23@example.com', ' mc23@supermail.eu']", ne gardons que marco23@example.com.
 - "nan" n'est pas un pays valide, nous le supprimons.
 - "24" n'est pas un pays valide, nous le supprimons.
153cm n'est pas au bon format. Il sera ignoré.
Imputation par la moyenne : 1.52
Imputation par la moyenne : 1.52


In [11]:
data

,prenom,email,date_naissance,pays,taille,prenom_min
0,Leila,leila@example.com,1990-01-23,France,1.49,leila
1,Samuel,samuel_329@example.com,2001-09-20,NaN,1.67,samuel
2,Radia,choupipoune@supermail.eu,NaT,Côte d'ivoire,1.52,radia
3,Marc,marco23@example.com,1978-02-10,France,1.65,marc
4,Heri,helloworld@supermail.eu,2008-03-05,Madagascar,1.34,heri
5,Hanna,hanna2019@supermail.eu,1970-01-01,NaN,1.52,hanna
6,samuël,samuel_329@example.com,NaT,Bénin,1.45,samuël
